In [3]:
import pandas as pd
import numpy as np
import shutil

# sci-kit learn
from sklearn.model_selection import StratifiedKFold

In [4]:
# Fold에 파일저장

df = pd.read_csv('./data/train.csv')
num_fold = 5
seed = 719

Kfold = StratifiedKFold(n_splits=num_fold, shuffle=True, random_state=seed)


for fold, (trn_idx, val_idx) in enumerate(Kfold.split(np.arange(df.shape[0]), df.label.values)):
    val = df.loc[val_idx, :]['image_id'].values
    for i, val in enumerate(val):
        #shutil.copyfile('./data/train_images/{}'.format(val), './data/train_images_relabeled/fold{}/{}'.format(fold, val))

FileNotFoundError: [Errno 2] No such file or directory: './data/train_images_relabeled/fold0/100042118.jpg'

In [5]:
for fold, (trn_idx, val_idx) in enumerate(Kfold.split(np.arange(df.shape[0]), df.label.values)):
    print(len(val_idx))

4280
4280
4279
4279
4279


In [12]:
for fold, (trn_idx, val_idx) in enumerate(Kfold.split(np.arange(df.shape[0]), df.label.values)):
    val = df.loc[val_idx, :]
    val.to_csv('./fold{}.csv'.format(fold), index=False)


In [11]:
for fold, (trn_idx, val_idx) in enumerate(Kfold.split(np.arange(df.shape[0]), df.label.values)):
    val = df.loc[val_idx, :]
    print(val)
    break

             image_id  label
2       100042118.jpg      1
15     1003218714.jpg      2
16     1003298598.jpg      3
21     1004163647.jpg      3
25     1004826518.jpg      2
...               ...    ...
21383   997161074.jpg      3
21385   997289539.jpg      2
21393   999329392.jpg      3
21394   999474432.jpg      1
21396   999998473.jpg      4

[4280 rows x 2 columns]


In [8]:
pred = [[0.1, 0.2, 0.6, 0.1], [0.2, 0.4, 0.1, 0.3]]

pred_1 = np.mean(pred, axis=0)

if max(pred_1) < 0.9:
    pred_1 = [np.nan]
    
pred_1

[nan]

In [9]:
pred_1[:]

[nan]

In [10]:
a= [[0.00303988 0.00445192 0.14309295 0.00346654 0.09594872]]
a = list(a)
a.shape

SyntaxError: invalid syntax (<ipython-input-10-244a4231b8b1>, line 1)

In [11]:
import os
import sys
import numpy as np
import pandas as pd
import shutil

import torch
from torchvision import transforms
import cv2

import albumentations as A

from datetime import datetime

## from tensor to numpy function
fn_tonumpy = lambda x: x.to('cpu').detach().numpy()

## 리스트 펼치기 함수
def flatten(lst):
    result = []
    for item in lst:
        result.extend(item)
    return result

## 제출파일 저장하기
def save_submission(data_dir, result_dir, pred, epoch, batch):
    submission = pd.DataFrame()
    submission['image_id'] = list(os.listdir(data_dir))
    submission['label'] = pred
    submission.to_csv('./result/submission.csv', index=False)


## 네트워크 불러오기
def load(ckpt_dir, net, optim):
    if not os.path.exists(ckpt_dir):
        epoch = 0
        return net, optim, epoch

    ckpt_lst = os.listdir(ckpt_dir)
    ckpt_lst.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

    dict_model = torch.load('%s/%s' % (ckpt_dir, ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])
    #epoch.load_state_dict(dict_model['epoch'])
    epoch = int(ckpt_lst[-1].split('epoch')[1].split('_batch')[0])

    return net, optim, epoch


In [12]:
import os
import sys
import numpy as np
import cv2

import torch

##
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, img_x, img_y, transform=None):
        self.data_dir = data_dir
        self.img_x = img_x
        self.img_y = img_y
        self.transform = transform

        lst_input = os.listdir(self.data_dir)
        self.lst_input = lst_input

    def __len__(self):
        return len(self.lst_input)

    def __getitem__(self, index):
        input = cv2.imread(os.path.join(self.data_dir, self.lst_input[index]), cv2.IMREAD_COLOR)
        input = cv2.cvtColor(input, cv2.COLOR_BGR2RGB)  # result of input shape is y,x,c

        if self.transform:
            input = self.transform(image=input)['image']

        data = {'input': input}

        return data

In [13]:
import timm

import torch
import torch.nn as nn

##
class CassvaImgClassifier(nn.Module):
    def __init__(self, network, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(network, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)

        '''
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.3),
            #nn.Linear(n_features, hidden_size,bias=True), nn.ELU(),
            nn.Linear(n_features, n_class, bias=True)
        )
        '''

    def forward(self, x):
        x = self.model(x)
        return x

In [14]:
import os
import sys
import numpy as np
import pandas as pd

# pytorch
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.cuda.amp.autocast_mode import autocast
from torch.cuda.amp import GradScaler

# sci-kit learn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import albumentations as A
from albumentations.pytorch import ToTensorV2

def transform_test(img_x, img_y):
    return A.Compose([
            A.RandomResizedCrop(img_x, img_y),
            A.Transpose(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0)], p=1.)

def dataloader_test(data_dir, img_x, img_y, batch_size):
    dataset_test = TestDataset(data_dir=data_dir, img_x=img_x, img_y=img_y, transform=transform_test(img_x, img_y))
    loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, num_workers=8)

    num_data_test = len(dataset_test)
    num_batch_test = np.ceil(num_data_test / batch_size)

    return loader_test, num_batch_test

In [32]:
def test(args):
    # hyperparameters
    img_x = args.img_x
    img_y = args.img_y

    lr = args.lr
    batch_size = args.batch_size

    num_epoch = args.num_epoch

    data_dir = args.data_dir
    ckpt_dir = args.ckpt_dir
    result_dir = args.result_dir

    network = args.network

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

##
    print("Test start ... ")
    df = pd.read_csv('./data/train.csv')

    loader_test, num_batch_test = dataloader_test(data_dir=data_dir, img_x=img_x, img_y=img_y, batch_size=batch_size)
    net = CassvaImgClassifier(network, df.label.nunique(), pretrained=False).to(device)
    optim = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=1e-6)
    net, optim, epoch = load(ckpt_dir=ckpt_dir, net=net, optim=optim)

    #used_epoch = [6, 7, 8, 9]  if test use rnd_epochs, it can have weights for each iteration.

    with torch.no_grad():
        net.eval()
        st_iter = 0
        tta = 1
        pred_tta = []

        for iter in range(st_iter + 1, tta + 1):
            pred_epoch = []

            for batch, data in enumerate(loader_test, 1):
                # forward pass
                input = data['input'].to(device)

                output = net(input)
                output = torch.softmax(output, dim=1).cpu().detach().numpy()

                print("TTA ITERATION: {}/{} | ".format(iter, tta), "BATCH: %04d / %04d" % (batch, num_batch_test))

                pred_epoch += [output]
                

            pred_epoch = np.concatenate(pred_epoch, axis=0)
            pred_tta += [pred_epoch/tta]
            
        pred = (np.mean(pred_tta, axis=0)).argmax(axis=1)
        # submission
        if tta % iter == 0:
            save_submission(data_dir=data_dir, result_dir=result_dir, pred=pred, epoch=num_epoch, batch=batch_size)

In [ ]:
import os
import numpy as np
import random

import torch

import argparse



## Parser 생성하기
parser = argparse.ArgumentParser(description="Cassava Leaf Disease Classification",
                                 formatter_class=argparse.ArgumentDefaultsHelpFormatter)

parser.add_argument("--mode", default="test", choices=["train", "test"], type=str, dest="mode")
parser.add_argument("--train_continue", default="off", choices=["on", "off"], type=str, dest="train_continue")

parser.add_argument("--seed", default=719, type=int, dest="seed")
parser.add_argument("--img_x", default=512, type=int, dest="img_x")
parser.add_argument("--img_y", default=512, type=int, dest="img_y")

parser.add_argument("--lr", default=1e-4, type=float, dest="lr")
parser.add_argument("--num_fold", default=10, type=int, dest="num_fold")
parser.add_argument("--num_epoch", default=10, type=int, dest="num_epoch")
parser.add_argument("--batch_size", default=8, type=int, dest="batch_size")

parser.add_argument("--cutmix", default=False, choices=[True, False], type=bool, dest="cutmix")
parser.add_argument("--fmix", default=False, choices=[True, False], type=bool, dest="fmix")

parser.add_argument("--label_smooth", default=False, choices=[True, False], type=bool, dest="label_smooth")
parser.add_argument("--swa", default=False, choices=[True, False], type=bool, dest="swa")

parser.add_argument("--data_dir", default="./data/val_images_relabeled/fold0", type=str, dest="data_dir")
parser.add_argument("--ckpt_dir", default="./checkpoint", type=str, dest="ckpt_dir")
parser.add_argument("--result_dir", default="./result", type=str, dest="result_dir")

parser.add_argument("--network", default="tf_efficientnet_b4_ns", type=str, dest="network")
parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")

args, unknown = parser.parse_known_args()

##
if __name__ == "__main__":
    # random seed
    def seed_everything(seed):
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

    seed = args.seed
    seed_everything(seed)

    if args.mode == "train":
        train(args)
    elif args.mode == "test":
        test(args)

Test start ... 
TTA ITERATION: 1/1 |  BATCH: 0001 / 0535
TTA ITERATION: 1/1 |  BATCH: 0002 / 0535
TTA ITERATION: 1/1 |  BATCH: 0003 / 0535
TTA ITERATION: 1/1 |  BATCH: 0004 / 0535
TTA ITERATION: 1/1 |  BATCH: 0005 / 0535
TTA ITERATION: 1/1 |  BATCH: 0006 / 0535
TTA ITERATION: 1/1 |  BATCH: 0007 / 0535
TTA ITERATION: 1/1 |  BATCH: 0008 / 0535
TTA ITERATION: 1/1 |  BATCH: 0009 / 0535
TTA ITERATION: 1/1 |  BATCH: 0010 / 0535
TTA ITERATION: 1/1 |  BATCH: 0011 / 0535
TTA ITERATION: 1/1 |  BATCH: 0012 / 0535
TTA ITERATION: 1/1 |  BATCH: 0013 / 0535
TTA ITERATION: 1/1 |  BATCH: 0014 / 0535
TTA ITERATION: 1/1 |  BATCH: 0015 / 0535
TTA ITERATION: 1/1 |  BATCH: 0016 / 0535
TTA ITERATION: 1/1 |  BATCH: 0017 / 0535
TTA ITERATION: 1/1 |  BATCH: 0018 / 0535
TTA ITERATION: 1/1 |  BATCH: 0019 / 0535
TTA ITERATION: 1/1 |  BATCH: 0020 / 0535
TTA ITERATION: 1/1 |  BATCH: 0021 / 0535
TTA ITERATION: 1/1 |  BATCH: 0022 / 0535
TTA ITERATION: 1/1 |  BATCH: 0023 / 0535
TTA ITERATION: 1/1 |  BATCH: 0024 / 0535


TTA ITERATION: 1/1 |  BATCH: 0202 / 0535
TTA ITERATION: 1/1 |  BATCH: 0203 / 0535
TTA ITERATION: 1/1 |  BATCH: 0204 / 0535
TTA ITERATION: 1/1 |  BATCH: 0205 / 0535
TTA ITERATION: 1/1 |  BATCH: 0206 / 0535
TTA ITERATION: 1/1 |  BATCH: 0207 / 0535
TTA ITERATION: 1/1 |  BATCH: 0208 / 0535
TTA ITERATION: 1/1 |  BATCH: 0209 / 0535
TTA ITERATION: 1/1 |  BATCH: 0210 / 0535
TTA ITERATION: 1/1 |  BATCH: 0211 / 0535
TTA ITERATION: 1/1 |  BATCH: 0212 / 0535
TTA ITERATION: 1/1 |  BATCH: 0213 / 0535
TTA ITERATION: 1/1 |  BATCH: 0214 / 0535
TTA ITERATION: 1/1 |  BATCH: 0215 / 0535
TTA ITERATION: 1/1 |  BATCH: 0216 / 0535
TTA ITERATION: 1/1 |  BATCH: 0217 / 0535
TTA ITERATION: 1/1 |  BATCH: 0218 / 0535
TTA ITERATION: 1/1 |  BATCH: 0219 / 0535
TTA ITERATION: 1/1 |  BATCH: 0220 / 0535
TTA ITERATION: 1/1 |  BATCH: 0221 / 0535
TTA ITERATION: 1/1 |  BATCH: 0222 / 0535
TTA ITERATION: 1/1 |  BATCH: 0223 / 0535
TTA ITERATION: 1/1 |  BATCH: 0224 / 0535
TTA ITERATION: 1/1 |  BATCH: 0225 / 0535
TTA ITERATION: 1